# Reproduction of Experiments on Dataset D22

## Read data

In [2]:
import sys 
import os
sys.path.insert(0, os.path.join("..", "src"))
import pandas as pd

path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
df = pd.read_csv(path, sep=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
label_column = "quality"

majority_labels = [5]
minority_labels = [3, 9]

df_filtered = df[df[label_column].isin(majority_labels + minority_labels)].copy()

df_filtered.loc[df_filtered[label_column].isin(majority_labels), label_column] = 1
df_filtered.loc[df_filtered[label_column].isin(minority_labels), label_column] = 0

X = df_filtered.drop(label_column, axis=1)
y = df_filtered[label_column]

In [4]:
y.value_counts()

1    1457
0      25
Name: quality, dtype: int64

# Classification

In [4]:
# adapted pipeline and custom transformer to support modifying both X and y (only on training data)
# adapted from: https://stackoverflow.com/questions/25539311/custom-transformer-for-sklearn-pipeline-that-alters-both-x-and-y
#from sklearn import pipeline
#from sklearn.base import clone
#from sklearn.utils import _print_elapsed_time
#from sklearn.utils.validation import check_memory
#from sklearn.pipeline import _fit_transform_one

#class Pipeline(pipeline.Pipeline):
#
#    def _fit(self, X, y=None, **fit_params_steps):
#        # shallow copy of steps - this should really be steps_
#        self.steps = list(self.steps)
#        self._validate_steps()
#        # Setup the memory
#        memory = check_memory(self.memory)
#
#        fit_transform_one_cached = memory.cache(_fit_transform_one)
#
#        for step_idx, name, transformer in self._iter(
#            with_final=False, filter_passthrough=False
#        ):
#            if transformer is None or transformer == "passthrough":
#                with _print_elapsed_time("Pipeline", self._log_message(step_idx)):
#                    continue
#
#            if hasattr(memory, "location") and memory.location is None:
#                # we do not clone when caching is disabled to
#                # preserve backward compatibility
#                cloned_transformer = transformer
#            else:
#                cloned_transformer = clone(transformer)
#            # Fit or load from cache the current transformer
#            X, fitted_transformer = fit_transform_one_cached(
#                cloned_transformer,
#                X,
#                y,
#                None,
#                message_clsname="Pipeline",
#                message=self._log_message(step_idx),
#                **fit_params_steps[name],
#            )
#            
#            if isinstance(X, tuple):    ###### unpack X if is tuple X = (X,y)
#                X, y = X
#            
#            # Replace the transformer of the step with the fitted
#            # transformer. This is necessary when loading the transformer
#            # from the cache.
#            self.steps[step_idx] = (name, fitted_transformer)
#        return X, y
#    
#    def fit(self, X, y=None, **fit_params):
#        """Fit the model.
#        Fit all the transformers one after the other and transform the
#        data. Finally, fit the transformed data using the final estimator.
#        Parameters
#        ----------
#        X : iterable
#            Training data. Must fulfill input requirements of first step of the
#            pipeline.
#        y : iterable, default=None
#            Training targets. Must fulfill label requirements for all steps of
#            the pipeline.
#        **fit_params : dict of string -> object
#            Parameters passed to the ``fit`` method of each step, where
#            each parameter name is prefixed such that parameter ``p`` for step
#            ``s`` has key ``s__p``.
#        Returns
#        -------
#        self : object
#            Pipeline with fitted steps.
#        """
#        fit_params_steps = self._check_fit_params(**fit_params)
#        Xt = self._fit(X, y, **fit_params_steps)
#        
#        if isinstance(Xt, tuple):    ###### unpack X if is tuple X = (X,y)
#            Xt, y = Xt 
#        
#        with _print_elapsed_time("Pipeline", self._log_message(len(self.steps) - 1)):
#            if self._final_estimator != "passthrough":
#                fit_params_last_step = fit_params_steps[self.steps[-1][0]]
#                self._final_estimator.fit(Xt, y, **fit_params_last_step)
#
#        return self
    
#class TrainDataAugmenter():
#    """Transformer that only applies augmentation to training data, not to test data"""
#    def __init__(self, augmentation_func, *arguments_for_augmentation_func):
#        self.augmentation_func = augmentation_func
#        self.arguments_for_augmentation_func = arguments_for_augmentation_func
#        
#    def fit(self, X, y=None):
#        print("Calling fit")
#        return self
#
#    def transform(self, X, y=None):
#        """This function is called on test data"""
#        print("Calling transform")
#        return X
#    
#    def fit_transform(self, X, y=None):
#        """This function is called on training data"""
#        print("Calling fit_transform")
#        X_augmented, y_augmented = self.augmentation_func(X, y, *self.arguments_for_augmentation_func)
#        return X_augmented, y_augmented

In [6]:
from sklearn.model_selection import ParameterGrid, KFold, GridSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np

> **Note:** In the following, we use a custom piece of code for the grid search and cross-validation. We do this to enforce that the augmentation/oversampling step is only applied to training data and the test data is actually kept separate. For the baseline model, this means that we could also simply make use of the functions provided by sklearn instead of our custom code. However, in order to make the results comparable, we use the same code in each case.

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=7018321)

## Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
rf_param_grid = {
    'n_estimators': [50, 100, 200, 400, 600],
    'max_depth': [None, 4, 6, 10, 20, 30, 50, 80, 100],
}

# compute all combinations of parameters
combination_dicts = list(ParameterGrid(rf_param_grid))

### Baseline (no augmentation)

In [12]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train, y_train)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [13]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': None, 'n_estimators': 200}
Best AUC: 0.8323561402985022


To verify that our code works as expected, we can compare to a regular sklearn grid search here:

In [11]:
baseline_search = GridSearchCV(RandomForestClassifier(random_state=19231823), rf_param_grid, scoring="roc_auc")
baseline_search.fit(X, y)
baseline_search.best_score_

0.8398663089017558

We can see that the best model found by a sklearn grid search performs similarly well to our best model, which indicates that the code works as expected.

### With CFA

In [14]:
from cfa import Iterative_CFA

For CFA, we try two different approaches, since it is not entirely clear from the paper what the authors actually did.
1. The first approach strictly follows the pseudo-code and description given in section 3.2 of the paper. 
2. In an earlier section of the paper, the authors say that _"the class of [a] new [synthetic counterfactual] instance needs to be verified by the underlying ML model."_ This means that we use some ML model trained on the data (without CFA) to assign a class to a new synthetic counterfactual, and only keep those that were classified to be a minority instance. This seems to lead to much more reasonable synthetic counterfactuals (see visualizations of the algorithm in exp001) but, at the same time, often means that the algorithm is unable to produce a fully balanced dataset (since it may terminate early).

> **Note:** We also use a different tolerance level of 50% here, since the 10%-threshold proposed by the authors does not yield any "good" native counterfactuals, which makes the algorithm unusable.

#### Approach 1: No verification 

In [15]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=False,
                                                         visualize_with_pca=False)
    
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Data distribution before CFA:
	Majority (1): 1166, Minority (label 0): 19
Number of 'good' native counterfactuals in data: 7
Data distribution after iteration 0:
	Majority (1): 1166, Minority (label 0): 1166
Data distribution after CFA:
	Majority (1): 1166, Minority (label 0): 1166
Fold 1:
Data distribution before CFA:
	Majority (1): 1162, Minority (label 0): 23
Number of 'good' native counterfactuals in data: 9
Data distribution after iteration 0:
	Majority (1): 1162, Minority (label 0): 1162
Data distribution after CFA:
	Majority (1): 1162, Minority (label 0): 1162
Fold 2:
Data distribution before CFA:
	Majority (1): 1168, Minority (label 0): 18
Number of 'good' native counterfactuals in data: 3
Data distribution after iteration 0:
	Majority (1): 1168, Minority (label 0): 1168
Data distribution after CFA:
	Majority (1): 1168, Minority (label 0): 1168
Fold 3:
Data distribution before CFA:
	Majority (1): 1164, Minority (label 0): 22
Number of 'good' native counterfactuals in da

In [14]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 10, 'n_estimators': 400}
Best AUC: 0.717917378820314


#### Approach 2: Verification of new synthetic counterfactuals with baseline model

In [17]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    print("Fitting baseline model needed for verification... ")
    baseline_search = GridSearchCV(RandomForestClassifier(random_state=19231823), rf_param_grid, scoring="roc_auc")
    baseline_search.fit(X_train, y_train)
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=True, 
                                                         baseline_model=baseline_search,
                                                         visualize_with_pca=False)
    
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 1166, Minority (label 0): 19
Number of 'good' native counterfactuals in data: 7
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 1166, Minority (label 0): 19
Fold 1:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 1162, Minority (label 0): 23
Number of 'good' native counterfactuals in data: 9
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 1162, Minority (label 0): 23
Fold 2:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 1168, Minority (label 0): 18
Number of 'good' native counterfactuals in data: 3
Classifier predicted all new synthetic c

In [16]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': None, 'n_estimators': 200}
Best AUC: 0.8323561402985022


### ADASYN instead of CFA

In [19]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=9317231)

In [20]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [19]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 20, 'n_estimators': 400}
Best AUC: 0.7789910523678737


### Random Oversampling

In [20]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=52012318)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [22]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': None, 'n_estimators': 600}
Best AUC: 0.7998980148267136


## Logistic Regression 

In [34]:
from sklearn.linear_model import LogisticRegression

C:\Users\fredd\AppData\Local\Temp\ipykernel_452\1504384812.py:6: DeprecationWarning: Please use `LineSearchWarning` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import LineSearchWarning


In [22]:
lr_param_grid = {"max_iter": [100, 200, 1000], 
                 "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                 "solver": ["newton-cg", "lbfgs", "liblinear", "sag"]}

# compute all combinations of parameters
combination_dicts = list(ParameterGrid(lr_param_grid))

### Baseline (no augmentation)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train, y_train)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [26]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 100, 'max_iter': 100, 'solver': 'newton-cg'}
Best AUC: 0.7668206151189778


Again, we can compare to an out-of-the-box grid search to make sure:

In [27]:
baseline_search = GridSearchCV(LogisticRegression(random_state=19231823), lr_param_grid, scoring="roc_auc")
baseline_search.fit(X, y)
baseline_search.best_score_

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the 

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarnin

0.7660349291531328

### With CFA

#### Approach 1: No verification 

In [35]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=False, 
                                                         visualize_with_pca=False)

    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Data distribution before CFA:
	Majority (1): 1166, Minority (label 0): 19
Number of 'good' native counterfactuals in data: 7
Data distribution after iteration 0:
	Majority (1): 1166, Minority (label 0): 1166
Data distribution after CFA:
	Majority (1): 1166, Minority (label 0): 1166
Fold 1:
Data distribution before CFA:
	Majority (1): 1162, Minority (label 0): 23
Number of 'good' native counterfactuals in data: 9
Data distribution after iteration 0:
	Majority (1): 1162, Minority (label 0): 1162
Data distribution after CFA:
	Majority (1): 1162, Minority (label 0): 1162
Fold 2:
Data distribution before CFA:
	Majority (1): 1168, Minority (label 0): 18
Number of 'good' native counterfactuals in data: 3
Data distribution after iteration 0:
	Majority (1): 1168, Minority (label 0): 1168
Data distribution after CFA:
	Majority (1): 1168, Minority (label 0): 1168
Fold 3:
Data distribution before CFA:
	Majority (1): 1164, Minority (label 0): 22
Number of 'good' native counterfactuals in da

In [29]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 0.01, 'max_iter': 200, 'solver': 'lbfgs'}
Best AUC: 0.7230129091143179


#### Approach 2: Verification of new synthetic counterfactuals with baseline model

In [36]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    print("Fitting baseline model needed for verification... ")
    baseline_search = GridSearchCV(LogisticRegression(random_state=19231823), lr_param_grid, scoring="roc_auc")
    baseline_search.fit(X_train, y_train)
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=True, 
                                                         baseline_model=baseline_search,
                                                         visualize_with_pca=False)

    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 1166, Minority (label 0): 19
Number of 'good' native counterfactuals in data: 7
Data distribution after iteration 0:
	Majority (1): 1166, Minority (label 0): 26
Number of 'good' native counterfactuals in data: 13
Data distribution after iteration 1:
	Majority (1): 1166, Minority (label 0): 30
Number of 'good' native counterfactuals in data: 17
Data distribution after iteration 2:
	Majority (1): 1166, Minority (label 0): 32
Number of 'good' native counterfactuals in data: 19
Data distribution after iteration 3:
	Majority (1): 1166, Minority (label 0): 34
Number of 'good' native counterfactuals in data: 21
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 1166, Minority (label 0): 34
Fold 1:
Fitting baseline model needed for verification... 
Data distribution bef

In [31]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 1000, 'max_iter': 100, 'solver': 'liblinear'}
Best AUC: 0.7630778556650009


### ADASYN

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [33]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 1, 'max_iter': 200, 'solver': 'lbfgs'}
Best AUC: 0.7471711051514929


### Random Oversampling

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [35]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 0.1, 'max_iter': 200, 'solver': 'lbfgs'}
Best AUC: 0.7633598537002654
